In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Data Preprocessing

english_sentences = ['start hello end', 'start world end', 'start how are you end', 'start I am fine end', 'start have a good day end']
spanish_sentences = ['start hola end', 'start mundo end', 'start cómo estás end', 'start estoy bien end', 'start ten un buen día end']

source_tokenizer = Tokenizer()
source_tokenizer.fit_on_texts(english_sentences)
source_sequences = source_tokenizer.texts_to_sequences(english_sentences)
input_texts = pad_sequences(source_sequences, padding='post')

target_tokenizer = Tokenizer()
target_tokenizer.fit_on_texts(spanish_sentences)
target_sequences = target_tokenizer.texts_to_sequences(spanish_sentences)
target_texts = pad_sequences(target_sequences, padding='post')


In [8]:
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Concatenate
from tensorflow.keras.layers import AdditiveAttention as Attention
from tensorflow.keras.models import Model

# Model parameters
embedding_dim = 256
latent_dim = 512
num_encoder_tokens = len(source_tokenizer.word_index) + 1
num_decoder_tokens = len(target_tokenizer.word_index) + 1

# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(num_encoder_tokens, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(num_decoder_tokens, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
# Attention Layer
attention = Attention()
attention_output = attention([decoder_outputs, encoder_outputs])

# Concatenating attention output and decoder LSTM output
decoder_concat_input = Concatenate(axis=-1)([decoder_outputs, attention_output])

# Dense layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_concat_input)

In [9]:
# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


In [10]:
from tensorflow.keras.utils import to_categorical
decoder_target_data = to_categorical(target_texts, num_decoder_tokens)
model.fit([input_texts, target_texts], decoder_target_data, batch_size=64, epochs=50, validation_split=0.2)


Epoch 1/50
1/1 [==============================] - 5s 5s/step - loss: 2.5697 - accuracy: 0.0417 - val_loss: 2.5720 - val_accuracy: 0.0000e+00
Epoch 2/50
1/1 [==============================] - 0s 159ms/step - loss: 2.4592 - accuracy: 0.4167 - val_loss: 2.5778 - val_accuracy: 0.0000e+00
Epoch 3/50
1/1 [==============================] - 0s 160ms/step - loss: 2.3456 - accuracy: 0.4167 - val_loss: 2.5890 - val_accuracy: 0.0000e+00
Epoch 4/50
1/1 [==============================] - 0s 171ms/step - loss: 2.1826 - accuracy: 0.4167 - val_loss: 2.6197 - val_accuracy: 0.0000e+00
Epoch 5/50
1/1 [==============================] - 0s 188ms/step - loss: 1.9245 - accuracy: 0.4167 - val_loss: 2.7407 - val_accuracy: 0.0000e+00
Epoch 6/50
1/1 [==============================] - 0s 165ms/step - loss: 1.6391 - accuracy: 0.4167 - val_loss: 2.8185 - val_accuracy: 0.0000e+00
Epoch 7/50
1/1 [==============================] - 0s 173ms/step - loss: 1.5187 - accuracy: 0.4167 - val_loss: 2.8343 - val_accuracy: 0.1667

In [11]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding_4 (Embedding)     (None, None, 256)            3840      ['input_5[0][0]']             
                                                                                                  
 embedding_5 (Embedding)     (None, None, 256)            3328      ['input_6[0][0]']             
                                                                                            